In [1]:
import json
import numpy as np
from scipy.spatial.transform import Rotation as R

with open("../drawing/drawing_config.json", "r") as f:
    config = json.load(f)

T, u, v, normal, origin = np.array(config["T"]), np.array(config["u"]), np.array(config["v"]), np.array(config["normal"]), np.array(config["origin"])

poses = config["poses"]
print("origin: ", origin)
print("u: ", u)
print("v: ", v)
print("normal: ", normal)
print("")

# pose positions
for i, s in enumerate(poses):
    pos = np.array([s["ee.x"], s["ee.y"], s["ee.z"]])
    rot = np.array([s["ee.wx"], s["ee.wy"], s["ee.wz"]])
    rot = R.from_euler('xyz', rot)
    R_matrix = rot.as_matrix()
    x_axis = R_matrix[:, 0]  # End-effector X axis in base frame
    y_axis = R_matrix[:, 1]  # End-effector Y axis
    z_axis = R_matrix[:, 2]  # End-effector Z axis

    tol = 1e-9
    print("Orthonormal check:",
          np.allclose(np.linalg.norm(x_axis), 1.0, atol=tol) and
          np.allclose(np.linalg.norm(y_axis), 1.0, atol=tol) and
          np.allclose(np.linalg.norm(z_axis), 1.0, atol=tol) and
          abs(np.dot(x_axis, y_axis)) < tol and
          abs(np.dot(y_axis, z_axis)) < tol and
          abs(np.dot(z_axis, x_axis)) < tol)

origin:  [0.19879036 0.01665487 0.06629053]
u:  [ 7.66542648e-01 -6.42193405e-01 -1.70412326e-06]
v:  [-6.42193405e-01 -7.66542648e-01 -2.03408522e-06]
normal:  [-7.04151223e-12  2.65358979e-06 -1.00000000e+00]

Orthonormal check: True
Orthonormal check: True
Orthonormal check: True
Orthonormal check: True
Orthonormal check: True
Orthonormal check: True
Orthonormal check: True


In [8]:

def plot_poses(T, u,v,origin, normal, poses):
    import plotly.graph_objects as go
    from scipy.spatial.transform import Rotation as R

    def arrow(start, vec, color, name=""):
        return go.Scatter3d(
            x=[start[0], start[0] + vec[0]],
            y=[start[1], start[1] + vec[1]],
            z=[start[2], start[2] + vec[2]],
            mode="lines+markers",
            line=dict(color=color, width=4),
            marker=dict(size=3, color=color),
            name=name
        )

    fig = go.Figure()

    # Axes
    fig.add_trace(arrow(origin, u * 0.01, "red", "u"))
    fig.add_trace(arrow(origin, v * 0.01, "green", "v"))
    fig.add_trace(arrow(origin, normal * 0.01, "black", "normal"))

    # pose positions
    for i, s in enumerate(poses):
        pos = np.array([s["ee.x"], s["ee.y"], s["ee.z"]])
        rot = np.array([s["ee.wx"], s["ee.wy"], s["ee.wz"]])
        rot = R.from_euler('xyz', rot)
        R_matrix = rot.as_matrix()
        x_axis = R_matrix[:, 0]  # End-effector X axis in base frame
        y_axis = R_matrix[:, 1]  # End-effector Y axis
        z_axis = R_matrix[:, 2]  # End-effector Z axis

        line_color = f"rgb({i * 15 % 255}, {i * 30 % 255}, 255)"
        fig.add_trace(arrow(pos, z_axis * 0.1, 'blue', "z"))
        fig.add_trace(arrow(pos, x_axis * 0.1, "red", "x"))
        fig.add_trace(arrow(pos, y_axis * 0.1, "green", "yaxis"))
        # --- Define rotation of +20° (in radians) about local x-axis ---
        angle_deg = -35
        angle_rad = np.deg2rad(angle_deg)
        R_local = R.from_rotvec(y_axis * angle_rad)  # rotate around local x-axis

        # --- Apply this local rotation to z-axis ---
        z_axis_rotated = R_local.apply(z_axis)

        # move along x_axis by 0.02m
        pos = pos + x_axis * 0.03
        pos = pos + z_axis * 0.07  # lift a bit
        # fig.add_trace(arrow(pos, z_axis_rotated * 0.05, line_color, "z-axis rotated"))
        #
        # fig.add_trace(
        #     go.Scatter3d(
        #         x=[pos[0]],
        #         y=[pos[1]],
        #         z=[pos[2]],
        #         mode="markers",
        #         marker=dict(size=4, color="blue"),
        #     )
        # )

    # Layout
    r = 1
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[origin[0] - r, origin[0] + r], title='X'),
            yaxis=dict(range=[origin[1] - r, origin[1] + r], title='Y'),
            zaxis=dict(range=[origin[2] - r, origin[2] + r], title='Z'),
            aspectmode='cube'
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        title="Interactive TCP Visualization",
    )
    fig.show()

print(len(poses))
plot_poses(T, u, v, origin, normal, poses)

7
